In [1]:
import selectors
import socket

mysel = selectors.DefaultSelector()
keep_running = True
outgoing = [
    b'It will be repeated.',
    b'This is the message.  ',
]
bytes_sent = 0
bytes_received = 0

# Connecting is a blocking operation, so call setblocking()
# after it returns.
server_address = ('localhost', 65535)
print('connecting to {} port {}'.format(*server_address))
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(server_address)
sock.setblocking(False)

# Set up the selector to watch for when the socket is ready
# to send data as well as when there is data to read.
mysel.register(
    sock,
    selectors.EVENT_READ | selectors.EVENT_WRITE,
)

while keep_running:
    print('waiting for I/O')
    for key, mask in mysel.select(timeout=1):
        print('key', key)
        print("mask", mask)
        connection = key.fileobj
        client_address = connection.getpeername()
        print('client({})'.format(client_address))

        print("EVENT_READ", selectors.EVENT_READ)
        print("mask & selectors.EVENT_READ", mask & selectors.EVENT_READ)
        if mask & selectors.EVENT_READ:
            print('  ready to read')
            data = connection.recv(1024)
            if data:
                # A readable client socket has data
                print('  received {!r}'.format(data))
                bytes_received += len(data)

            # Interpret empty result as closed connection,
            # and also close when we have received a copy
            # of all of the data sent.
            keep_running = not (
                data or
                (bytes_received and
                 (bytes_received == bytes_sent))
            )

        if mask & selectors.EVENT_WRITE:
            print('  ready to write')
            if not outgoing:
                # We are out of messages, so we no longer need to
                # write anything. Change our registration to let
                # us keep reading responses from the server.
                print('  switching to read-only')
                mysel.modify(sock, selectors.EVENT_READ)
            else:
                # Send the next message.
                next_msg = outgoing.pop()
                print('  sending {!r}'.format(next_msg))
                sock.sendall(next_msg)
                bytes_sent += len(next_msg)

print('shutting down')
mysel.unregister(connection)
connection.close()
mysel.close()

connecting to localhost port 65535
waiting for I/O
key SelectorKey(fileobj=<socket.socket fd=1120, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 64822), raddr=('127.0.0.1', 65535)>, fd=1120, events=3, data=None)
mask 2
client(('127.0.0.1', 65535))
EVENT_READ 1
mask & selectors.EVENT_READ 0
  ready to write
  sending b'This is the message.  '
waiting for I/O
key SelectorKey(fileobj=<socket.socket fd=1120, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 64822), raddr=('127.0.0.1', 65535)>, fd=1120, events=3, data=None)
mask 2
client(('127.0.0.1', 65535))
EVENT_READ 1
mask & selectors.EVENT_READ 0
  ready to write
  sending b'It will be repeated.'
waiting for I/O
key SelectorKey(fileobj=<socket.socket fd=1120, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 64822), raddr=('127.0.0.1', 65535)>, fd=1120, events=3, data=None)
mask 2
client(('127.0.0.1', 65535))
EVENT_READ